# 📌 상품 정보 크롤링  

### ✏️ 하위 카테고리 별 상품 링크 저장

In [13]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# 주소 담을 리스트
page_list = []

# small_categories.txt 파일 읽어오기
small_categories = "C:\SSAFY\자율프로젝트\자율크롤링\small_categories.txt"
# small_categories = "D:\SSAFY\자율\크롤링\자율크롤링\small_categories.txt"
f = open(small_categories, "r", encoding="UTF8")


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = strings[0]
    category_url = strings[1]
    print(category_uid)
    print(category_url)
    
    ## 크롤링
    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(category_url)
    web_driver.implicitly_wait(5)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    link_list = soup.find_all('a', class_='srchGridProductUnitLink',href=True)
    
    # print(link_list)
    # print(len(link_list))

#     목록에서 link 얻어오기 (2개씩 얻어와지기때문에 짝수번은 날려보내기)
    for index, list in enumerate(link_list):
        if index %2 == 0:
            page_list.append(category_uid + " " + list['href'])
        if index >= 39:
            break
            
time.sleep(5)

    # page_list 저장하기 
# fo = open("D:\SSAFY\자율\크롤링\자율크롤링\products.txt", "w", encoding="UTF8")
fo = open("C:\SSAFY\자율프로젝트\자율크롤링\products.txt", "w", encoding="UTF8")
for page in page_list:
    fo.write(page)
    fo.write("\n")
fo.close
    
print(len(page_list))

    
f.close


    



    


1
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&u9=navigate&u8=FC01200000&login=Y&mallId=1
2
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC01260000&mallId=1
3
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&u9=navigate&u8=FC01080000&login=Y&mallId=1
4
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC01040000&mallId=1
5
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC01220000&mallId=1
6
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC02080000&mallId=1
7
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC02090000&mallId=1
8
https://www.lotteon.com/search/render/render.ecn?rend

<function TextIOWrapper.close()>

### ✏️ 상품 정보 DB에 저장 (product, brand, picture)

In [18]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from openpyxl import Workbook
import pymysql

# DB 연결용
# conn = pymysql.connect(host='k7e201.p.ssafy.io',
#                        user='abuyz7e201',
#                        password='a-buy-z-e201',
#                        db='wkdbf',
#                        charset='utf8')

def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn

# small_categories.txt 파일 읽어오기
# products = "D:\SSAFY\자율\크롤링\자율크롤링\products.txt"
# products = "C:\SSAFY\자율프로젝트\자율크롤링\products.txt"
products = "C:\SSAFY\자율프로젝트\자율크롤링\products_temp.txt"
f = open(products, "r", encoding="UTF8")


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = strings[0]
    url = strings[1]
    print(category_uid)
    print(url)
    print("------------------------")

    # 임시 url
    # temp_url= "https://www.lotteon.com/p/product/LO1987818462?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC01200000&clickId=C33295679634"
#     temp_url = "https://www.lotteon.com/p/product/LO1566256308?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC04020000&clickId=C33403790992"

    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(url)
    web_driver.implicitly_wait(3)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # 데이터 찾기 
    product_name = soup.find('div', class_='productName').text
    des_picture = soup.find('iframe', class_='m2-prd-frame')['src']

    status = '새상품'

    ## 할인율 없으면
    if soup.find_all("span", {"class": "discountRate"}):
        discount_rate = soup.find('span', class_='discountRate').text.replace("할인율","").replace(" ","").replace("%","")
        product_price = soup.find('div', class_='listPrice').select_one(":nth-child(2)").text.replace(" ","").replace("\n","").replace(",","")
    else:
        discount_rate = 0
        product_price = soup.find('div', class_='price').text.replace(" ","").replace("\n","").replace("할인가","").replace("원","").replace(",","")

    # int값으로 변환
    product_price = int(product_price)
    discount_rate = int(discount_rate)

    # 리뷰가 있는지 확인
    if soup.find_all("a", {"class": "staring"}):
        review_rating = soup.find('a', class_='staring').em.text # 5.0
    else:
        review_rating = 0;
    delivery_fee = soup.find_all('li', class_='soonInfoList')[1].text

    # 배송비 무료 > int 변환
    val = "무료" in delivery_fee
    if val:
        delivery_fee = 0
    else:
        delivery_fee = 3000



    # 이미지(썸네일) 
    picture_list = soup.find('div', class_='productVisualThumbs')
#   rep_img =  picture_list.find('img')['src']
    rep_img = picture_list.find('img')['src']
#     print("rep_img",rep_img)
    rep_img_list =  picture_list.find_all('img')
#     for img in rep_img_list:
#         print(img['src'])


    ############### 브랜드 ################
    brand_uid = 0
    brand_isAlready = True
    if soup.find_all("div", {"class": "brand"}):
        brand_name = soup.find("div", {"class": "brand"}).text
        brand_url = soup.find("div", {"class": "brandImage"}).select_one(":nth-child(1)")['src']
        print(brand_name)
        # 해당 브랜드를 가지고 있는지 확인
        sql = "SELECT uid FROM brands WHERE name = %s"
        with connectiondb() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, brand_name)
                result = cur.fetchone()
                if result is None:
                    print("table에 브랜드없음")
                    brand_isAlready = False
                else: 
                    print("table에 브랜드있음")
                    brand_isAlready = True
                    brand_uid = result[0]
    else:
        print("브랜드없는상품")
        brand_uid =  1 # 브랜드 없음 


    if(brand_isAlready is False):
        print("브랜드생성")
        # 없으면 새로 만들고 uid 값 가져오기
        sql = "INSERT INTO brands (img_url, name) VALUES (%s, %s)"
        with connectiondb() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, (brand_url, brand_name))
                brand_uid = cur.lastrowid
                conn.commit()
    print("@@brand insert@@")

    ############### product insert ################
    # 형변환
    category_uid = int(category_uid)
    review_rating = float(review_rating)
    
    
#     print("delivery_fee",delivery_fee)
#     print("discount_rate",discount_rate)
#     print("price",product_price)
#     print("name",product_name)
#     print("description_img",des_picture)
#     print("rep_img",rep_img)
#     print("review_rate",review_rating)
#     print("status",status)
#     print("brand_uid",brand_uid)
#     print("small_categories_uid", category_uid)
    
    
#     print(type(delivery_fee))
#     print(type(des_picture))
#     print(type(discount_rate))
#     print(type(product_name))
#     print(type(product_price))
#     print(type(rep_img))
#     print(type(review_rating))
#     print(type(status))
#     print(type(brand_uid))
#     print(type(category_uid))
    

    sql = "INSERT INTO products (delivery_fee, description_img, discount_rate, name, price, rep_img, review_rate, status, brands_uid, small_categories_uid) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    with connectiondb() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (delivery_fee, des_picture, discount_rate, product_name, product_price, rep_img, review_rating, status, brand_uid, category_uid))
            product_uid = cur.lastrowid
            conn.commit()
            
    print("@@product insert@@")


    ############### 이미지 insert ################
    ## insert 이후 가장 최근 product uid 데려와서 상품 사진 테이블에 insert 하기 
    min_length = min(len(rep_img_list),3)
#     print("min_length",min_length)
    
    for i in range(min_length):
        product_thumbnail = rep_img_list[i]['src']
#         print("product_thumbnail",product_thumbnail)
    
        sql = "INSERT INTO product_pictures (img_url,products_uid) VALUES (%s,%s)"
        with connectiondb() as conn: 
            with conn.cursor() as cur:
                cur.execute(sql, (product_thumbnail, product_uid))
                conn.commit()
    print("@@image insert@@")
    ############### 옵션 insert ################
    # 워크벤치에서 따로 
    
    print("====================================================")

    

f.close


25
https://www.lotteon.com/p/product/LO1645491725?sitmNo=LO1645491725_1645491726&mall_no=1&dp_infw_cd=CASfc04020000
------------------------
케라셀
table에 브랜드없음
브랜드생성
@@brand insert@@
@@product insert@@
@@image insert@@
25
https://www.lotteon.com/p/product/LO1905915980?sitmNo=LO1905915980_1905915981&mall_no=1&dp_infw_cd=CASFC04020000
------------------------
공식 브랜드관베스트 브랜드에뛰드
table에 브랜드있음
@@brand insert@@
@@product insert@@
@@image insert@@
25
https://www.lotteon.com/p/product/LO1819777541?sitmNo=LO1819777541_1819777543&mall_no=1&dp_infw_cd=CASFC04020000
------------------------
브랜드없는상품
@@brand insert@@
@@product insert@@
@@image insert@@


<function TextIOWrapper.close()>

### ✏️ 옵션 넣기

In [103]:
import requests
import re
import os
import pymysql


def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn


## 1. 과일
sql = "select uid from wkdbf.products where small_categories_uid = 38"
with connectiondb() as conn: 
    with conn.cursor() as cur:
        cur.execute(sql)
        result = cur.fetchall()

        
for temp in result:
    print(temp[0])

    sql = "insert into wkdbf.product_options (name, value, products_uid) values (%s,%s,%s);"
    with connectiondb() as conn: 
        with conn.cursor() as cur:
            cur.execute(sql, ('color', '블랙', temp[0]))
            conn.commit()

512


### ✏️ 추가 카테고리 데이터 넣기(카테고리당 한개)  

In [7]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import pymysql

# DB 연결용
# conn = pymysql.connect(host='k7e201.p.ssafy.io',
#                        user='abuyz7e201',
#                        password='a-buy-z-e201',
#                        db='wkdbf',
#                        charset='utf8')

def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn





# 임시 url
temp_url= "https://www.lotteon.com/p/product/LO1532037525?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC08070000&clickId=C33458657195"

category_uid = 39


webdriver_options=webdriver.ChromeOptions()
webdriver_options.add_argument("--start-maximized")
web_driver = webdriver.Chrome(options=webdriver_options)
web_driver.get(temp_url)
web_driver.implicitly_wait(3)
html = web_driver.page_source
soup = BeautifulSoup(html, 'html.parser')


    # 데이터 찾기 
product_name = soup.find('div', class_='productName').text
des_picture = soup.find('iframe', class_='m2-prd-frame')['src']

status = '새상품'

    ## 할인율 없으면
if soup.find_all("span", {"class": "discountRate"}):
    discount_rate = soup.find('span', class_='discountRate').text.replace("할인율","").replace(" ","").replace("%","")
    product_price = soup.find('div', class_='listPrice').select_one(":nth-child(2)").text.replace(" ","").replace("\n","").replace(",","")
else:
    discount_rate = 0
    product_price = soup.find('div', class_='price').text.replace(" ","").replace("\n","").replace("할인가","").replace("원","").replace(",","")

    # int값으로 변환
product_price = int(product_price)
discount_rate = int(discount_rate)

    # 리뷰가 있는지 확인
if soup.find_all("a", {"class": "staring"}):
    review_rating = soup.find('a', class_='staring').em.text # 5.0
else:
    review_rating = 0;
delivery_fee = soup.find_all('li', class_='soonInfoList')[1].text

    # 배송비 무료 > int 변환
val = "무료" in delivery_fee
if val:
    delivery_fee = 0
else:
    delivery_fee = 3000



    # 이미지(썸네일) 
picture_list = soup.find('div', class_='productVisualThumbs')
#   rep_img =  picture_list.find('img')['src']
rep_img = picture_list
#     print("rep_img",rep_img)
rep_img_list =  picture_list.find_all('img')
for img in rep_img_list:
    print(img['src'])
main_img = soup.find('div', class_='largeImgSlide').find('img')['src']
print("메인썸네일", main_img)

    ############### 브랜드 ################
brand_uid = 0
brand_isAlready = True
if soup.find_all("div", {"class": "brand"}):
    brand_name = soup.find("div", {"class": "brand"}).text
    brand_url = soup.find("div", {"class": "brandImage"}).select_one(":nth-child(1)")['src']
    print(brand_name)
        # 해당 브랜드를 가지고 있는지 확인
    sql = "SELECT uid FROM brands WHERE name = %s"
    with connectiondb() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, brand_name)
            result = cur.fetchone()
            if result is None:
                print("table에 브랜드없음")
                brand_isAlready = False
            else: 
                print("table에 브랜드있음")
                brand_isAlready = True
                brand_uid = result[0]
else:
    print("브랜드없는상품")
    brand_uid =  1 # 브랜드 없음 


if(brand_isAlready is False):
    print("브랜드생성")
        # 없으면 새로 만들고 uid 값 가져오기
    sql = "INSERT INTO brands (img_url, name) VALUES (%s, %s)"
    with connectiondb() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (brand_url, brand_name))
            brand_uid = cur.lastrowid
            conn.commit()
print("@@brand insert@@")

    ############### product insert ################
    # 형변환
category_uid = int(category_uid)
review_rating = float(review_rating)


print("delivery_fee",delivery_fee)
print("discount_rate",discount_rate)
print("price",product_price)
print("name",product_name)
print("description_img",des_picture)
print("rep_img",rep_img)
print("review_rate",review_rating)
print("status",status)
print("brand_uid",brand_uid)
print("small_categories_uid", category_uid)
    

sql = "INSERT INTO products (delivery_fee, description_img, discount_rate, name, price, rep_img, review_rate, status, brands_uid, small_categories_uid) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
# with connectiondb() as conn:
#     with conn.cursor() as cur:
#         cur.execute(sql, (delivery_fee, des_picture, discount_rate, product_name, product_price, rep_img, review_rating, status, brand_uid, category_uid))
#         product_uid = cur.lastrowid
#         conn.commit()
            
print("@@product insert@@")


    ############### 이미지 insert ################
    ## insert 이후 가장 최근 product uid 데려와서 상품 사진 테이블에 insert 하기 
min_length = min(len(rep_img_list),3)
#     print("min_length",min_length)
    
for i in range(min_length):
    product_thumbnail = rep_img_list[i]['src']
#         print("product_thumbnail",product_thumbnail)
    
#     sql = "INSERT INTO product_pictures (img_url,products_uid) VALUES (%s,%s)"
#     with connectiondb() as conn: 
#         with conn.cursor() as cur:
#             cur.execute(sql, (product_thumbnail, product_uid))
#             conn.commit()
print("@@image insert@@")
    ############### 옵션 insert ################
    # 워크벤치에서 따로 
    
print("====================================================")



https://contents.lotteon.com/itemimage/_v125620/LO/15/32/03/75/25/_1/53/20/37/52/6/LO1532037525_1532037526_1.jpg/dims/resizef/67x67
메인썸네일 https://contents.lotteon.com/itemimage/_v125620/LO/15/32/03/75/25/_1/53/20/37/52/6/LO1532037525_1532037526_1.jpg/dims/resizef/554X554
카르닉
table에 브랜드있음
@@brand insert@@
delivery_fee 0
discount_rate 0
price 47900
name 1+1 카르닉 폴딩 캠핑의자 (블랙) 야외 낚시 폴딩의자
description_img https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F15%2F32%2F03%2F75%2F25%2FDSCRP_LO1532037525
rep_img <div class="swiper-container productVisualThumbs swiper-container-initialized swiper-container-vertical" data-v-02fae411=""> <div class="swiper-wrapper" style="transition-duration: 0ms; transform: translate3d(0px, 159px, 0px);"><div class="swiper-slide swiper-slide-active" data-v-02fae411="" style="margin-bottom: 10px;"><div class="imgWrap" data-cmpnt-name="pdd_thumb_preview_img" data-cmpnt-typ="pdd_basicinfo" data-gtm-vis-first-on-screen-11901096_2002="1437" data-gt

### ✏️ 메인 이미지 사진 변경하기 

In [18]:
import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import pymysql

# DB 연결용
# conn = pymysql.connect(host='k7e201.p.ssafy.io',
#                        user='abuyz7e201',
#                        password='a-buy-z-e201',
#                        db='wkdbf',
#                        charset='utf8')

def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn

products = "C:\SSAFY\자율프로젝트\자율크롤링\products_temp.txt"
f = open(products, "r", encoding="UTF8")
    
# 22부터 513까지 
uid = 145 부터 


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = strings[0]
    url = strings[1]
    print(category_uid)
    print(url)
    print("@@@@@@@@@@uid : ", uid)
    print("------------------------")

    sql = "select * from wkdbf.products where uid = %s"
    with connectiondb() as conn: 
        with conn.cursor() as cur:
            cur.execute(sql, uid)
            result = cur.fetchall()
    print(result)
    
    


    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(url)
    web_driver.implicitly_wait(3)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # 메인 상품 이미지
    main_img = soup.find('div', class_='largeImgSlide').find('img')['src']
    print("@@@@@@@@@@메인썸네일 : ", main_img)


    sql = "UPDATE wkdbf.products SET rep_img=%s WHERE uid=%s"
    with connectiondb() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (main_img, uid))
            conn.commit()
            
    uid += 1
    if uid == 25 or uid == 75:
        uid += 1
        
    if(uid == 505):
        uid += 2
    
    print("===============================================")

f.close

3
https://www.lotteon.com/p/product/bundle/LO1857306368?sitmNo=&mall_no=1&dp_infw_cd=CASFC01080000
@@@@@@@@@@uid :  111
------------------------
((111, datetime.datetime(2022, 11, 3, 1, 16, 48), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F17%2F62%2F95%2F26%2F02%2FDSCRP_LO1762952602', 6, '(본사직영)코카콜라 190ml캔 30입', 21500, 'https://contents.lotteon.com/itemimage/_v131551/LO/17/62/95/26/02/_1/76/29/52/60/3/LO1762952602_1762952603_1.jpg/dims/resizef/67x67', 4.8, '새상품', 55, 5, None),)
@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/LO/18/57/30/63/68/bndl_img/LO1857306368_1.jpg/dims/resizef/554X554
3
https://www.lotteon.com/p/product/LM2861840000005?sitmNo=LM2861840000005_001&mall_no=1&dp_infw_cd=CASFC01080000
@@@@@@@@@@uid :  112
------------------------
((112, datetime.datetime(2022, 11, 3, 1, 16, 55), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLD%2F76%2F21%2F27%2F66%2F4%2FDSCRP_LD762127664', 6, '(본사직영)코크제로 355m

((121, datetime.datetime(2022, 11, 3, 1, 18, 3), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F14%2F59%2F62%2F99%2F44%2FDSCRP_LO1459629944', 6, '(본사직영) 토레타 무라벨 340ml PET 20입', 17600, 'https://contents.lotteon.com/itemimage/_v101841/LO/14/59/62/99/44/_1/45/96/29/94/5/LO1459629944_1459629945_1.png/dims/resizef/67x67', 4.9, '새상품', 55, 5, None),)
@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/LO/15/87/32/78/33/_1/58/73/27/83/4/LO1587327833_1587327834_1.jpg/dims/resizef/554X554
4
https://displayad.lotteon.com/cadClick?x=578LfdQlXp0%2F6P%2B%2B3Tf%2FamDDaAAcOtvSJeZIUNfUwvJghgF3PBBD2RKalF%2BkfUybehi1pxrvN%2BR1p5lQsa%2F9afkk7wX97UF0AKsLN1SLNOVNslG4N2o746Kp4i8P7I1lWiDpTAcj0Zk%2BV1yVpDM%2B7WzXW3PN23%2BwolDJ%2F64CACDZtVFaEjcrLqmIyq76FsjUXtuAeWl8bV%2B6BQwd2iMMeaFdjfQ437hW7QhYfNluyCB3BXln%2BJd5tAD9NTRMk04YW%2BHOJoujN8doKxsJwNG5kQ%3D%3D&fwurl=https%3A%2F%2Fwww.lotteon.com%2Fp%2Fproduct%2FLO1592663660%3FareaCode%3DAD_CPC%26entryPoint%3Dad%26dp_infw_cd%3DC

@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v174317/LO/18/72/32/44/37/_1/87/23/24/43/8/LO1872324437_1872324438_1.jpg/dims/resizef/554X554
4
https://www.lotteon.com/p/product/LM8801019610783?sitmNo=LM8801019610783_001&mall_no=1&dp_infw_cd=CASFC01040000
@@@@@@@@@@uid :  132
------------------------
((132, datetime.datetime(2022, 11, 3, 1, 19, 38), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F10%2F00%2F23%2F32%2F34%2FDSCRP_LO1000233234', 20, '페브리즈 공기탈취제 화장실용 비치형 6ml 4개(2+2)', 23300, 'https://contents.lotteon.com/itemimage/LO/10/00/23/32/34/_1/02/44/65/22/9/LO1000233234_1024465229_1.jpg/dims/resizef/67x67', 4.8, '새상품', 60, 6, None),)
@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v070712/LM/88/01/01/96/10/78/3_/00/1/LM8801019610783_001_1.jpg/dims/resizef/554X554
4
https://www.lotteon.com/p/product/PD24043561?mall_no=1&dp_infw_cd=CASFC01040000
@@@@@@@@@@uid :  133
------------------------
((133, datetime.datetime(2022, 11, 3, 1

@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v223606/LD/75/87/42/86/9_/0/LD758742869_0_2.jpg/dims/resizef/554X554
5
https://displayad.lotteon.com/cadClick?x=dShNcUvzMdGLXfXW6%2BeZZkBmyYW73hF4FHAh%2Fqbt8z7vznM7yy9lXkeooRvdgQGIYsRD%2BuJOFWlMcsDUoaK6nTGQUq84hM4vXFsfXepNNHM0wphrFINCuP%2FjPctj2FiwYXMCzVSl0buzSbdxVSE0RcFCZOKrFjFUGQd7tLHQrkIeVosA0fVm6Uo8Tz%2FAVb%2B5fkgMpSlXgpgo7qrvuoZqPtXrkiDHoLiSckmpTc6jpsb0Hrl2E6Zh6OLljo7nJKhzEw1AWWFDh9xLNCSVtHhIpg%3D%3D&fwurl=https%3A%2F%2Fwww.lotteon.com%2Fp%2Fproduct%2FLO1609494903%3FareaCode%3DAD_CPC%26entryPoint%3Dad%26dp_infw_cd%3DCASFC01220000
@@@@@@@@@@uid :  142
------------------------
((142, datetime.datetime(2022, 11, 3, 1, 21, 12), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F12%2F76%2F82%2F11%2F67%2FDSCRP_LO1276821167', 20, '[페브리즈]섬유탈취제/공기탈취제/차랑용/욕실용 방향제 BEST', 19900, 'https://contents.lotteon.com/itemimage/LO/14/51/46/85/29/bndl_img/LO1451468529_1.jpg/dims/resizef/67x67', 4.8, '새상품', 60, 6, N

AttributeError: 'NoneType' object has no attribute 'find'